# Regulon scores

In [ ]:
options(warn=-1)

In [ ]:
library_load <- suppressMessages(
    
    suppressWarnings(
        
        list(
        
            # Seurat 
            library(Seurat), 

            # Data 
            library(tidyverse), 
            library(data.table), 

            # Plotting 
            library(patchwork), 
            library(cowplot), 
            library(ggrepel), 

            # Pyhton compatibility
            library(reticulate)

        )
    )
)

In [ ]:
# Configure reticulate 
use_condaenv(condaenv="p.3.9.19-FD20200109SPLENO", conda="/nobackup/peer/fdeckert/miniconda3/bin/conda", required=NULL)
py_config()

In [ ]:
random_seed <- 42
set.seed(random_seed)

In [ ]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [ ]:
# Source
source("bin/so_pl.R")

In [ ]:
# Plotting Theme
source("plotting_global.R")
ggplot2::theme_set(theme_global_set(size_select=1)) # From project global source()

# Import regulon AUC scores 

In [ ]:
# Import per cell regulon auc scores 
auc_mtx <- read.csv("/research/peer/fdeckert/FD20200109SPLENO/result/scenic/scRNAseq/auc_mtx.csv", row.names=1)
colnames(auc_mtx) <- gsub("\\.", "", colnames(auc_mtx))

In [ ]:
# Import Seurat 
so <- readRDS("data/BSA_0355_SM01_10x_SPLENO/ANALYSIS/pp.rds")

In [ ]:
# Add auc assay
so[["AUC"]] <- CreateAssayObject(counts=t(auc_mtx))

# Import regulon specific score (RSS)

In [ ]:
rss_baseline_mtx <- read.csv("/research/peer/fdeckert/FD20200109SPLENO/result/scenic/scRNAseq/rss_baseline_mtx.csv", row.names=1) %>% tibble::rownames_to_column("leiden_cell_type_main")
colnames(rss_baseline_mtx) <- gsub("\\.", "", colnames(rss_baseline_mtx))

In [ ]:
rss_baseline_mat <- reshape2::melt(rss_baseline_mtx, id.vars=c("leiden_cell_type_main"), variable.name="module", value.name="rss") %>% 
    
    dplyr::arrange(leiden_cell_type_main, desc(rss)) %>% 
    dplyr::group_by(leiden_cell_type_main) %>%
    dplyr::mutate(rank=1:n()) %>% 
    dplyr::mutate(label_rank=ifelse(rank %in% 1:15, as.character(module), NA)) %>% 
    dplyr::mutate(leiden_cell_type_main=factor(leiden_cell_type_main, levels=names(color$leiden_cell_type_main)))

In [ ]:
options(repr.plot.width=25, repr.plot.height=10)

dp_rss_top_rank <- ggplot(rss_baseline_mat, aes(x=rank, y=rss, label=label_rank, color=leiden_cell_type_main)) + 
    geom_point(size=3) + 
    xlab("Rank") + ylab("RSS") + ggtitle("RSS (Baseline)") + 
    ylim(c(min(rss_baseline_mat$rss), max(rss_baseline_mat$rss) + 0.1)) + 
    facet_wrap(~leiden_cell_type_main, scales="free_y", ncol=10) +
    scale_color_manual(values=color$leiden_cell_type_main) + 
    scale_x_continuous(breaks=seq(0, floor(max(rss_baseline_mat$rank)/100)*100, 100), labels=seq(0, floor(max(rss_baseline_mat$rank)/100)*100, 100)) + 
    geom_text_repel(segment.color="black", force=10, force_pull=1, max.overlaps=getOption("ggrepel.max.overlaps", default=100), size=5, alpha=1, guide="none", segment.size=0.05, color="black", min.segment.length=0) + 
    theme(legend.position="none") + 
    guides(color=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0, keyheight=0.25, default.unit="cm"))

dp_rss_top_rank

In [ ]:
rss_cpg_mtx <- read.csv("/research/peer/fdeckert/FD20200109SPLENO/result/scenic/scRNAseq/rss_cpg_mtx.csv", row.names=1) %>% tibble::rownames_to_column("leiden_cell_type_main")
colnames(rss_cpg_mtx) <- gsub("\\.", "", colnames(rss_cpg_mtx))

In [ ]:
rss_cpg_mat <- reshape2::melt(rss_cpg_mtx, id.vars=c("leiden_cell_type_main"), variable.name="module", value.name="rss") %>% 
    
    dplyr::arrange(leiden_cell_type_main, desc(rss)) %>% 
    dplyr::group_by(leiden_cell_type_main) %>%
    dplyr::mutate(rank=1:n()) %>% 
    dplyr::mutate(label_rank=ifelse(rank %in% 1:15, as.character(module), NA)) %>% 
    dplyr::mutate(leiden_cell_type_main=factor(leiden_cell_type_main, levels=names(color$leiden_cell_type_main)))

In [ ]:
options(repr.plot.width=25, repr.plot.height=10)

dp_rss_top_rank <- ggplot(rss_cpg_mat, aes(x=rank, y=rss, label=label_rank, color=leiden_cell_type_main)) + 
    geom_point(size=3) + 
    xlab("Rank") + ylab("RSS") + ggtitle("RSS (CpG)") + 
    ylim(c(min(rss_baseline_mat$rss), max(rss_baseline_mat$rss) + 0.1)) + 
    facet_wrap(~leiden_cell_type_main, scales="free_y", ncol=10) +
    scale_color_manual(values=color$leiden_cell_type_main) + 
    scale_x_continuous(breaks=seq(0, floor(max(rss_baseline_mat$rank)/100)*100, 100), labels=seq(0, floor(max(rss_baseline_mat$rank)/100)*100, 100)) + 
    geom_text_repel(segment.color="black", force=10, force_pull=1, max.overlaps=getOption("ggrepel.max.overlaps", default=100), size=5, alpha=1, guide="none", segment.size=0.05, color="black", min.segment.length=0) + 
    theme(legend.position="none") + 
    guides(color=guide_legend(ncol=1, override.aes=list(alpha=1, size=1.5), keywidth=0, keyheight=0.25, default.unit="cm"))

dp_rss_top_rank

## Compare regulon score between treatment groups

In [ ]:
rss_mat <- dplyr::inner_join(
    
    dplyr::select(rss_cpg_mat, leiden_cell_type_main, module, rss) %>% dplyr::rename(rss_cpg=rss), 
    dplyr::select(rss_baseline_mat, leiden_cell_type_main, module, rss) %>% dplyr::rename(rss_baseline=rss), 
    by=join_by(leiden_cell_type_main, module)
    
)

In [ ]:
options(repr.plot.width=25, repr.plot.height=10)

ggplot(rss_mat, aes(x=rss_baseline, y=rss_cpg, color=leiden_cell_type_main)) + 
    geom_abline(intercept=0, slope=1, linetype = "dashed") + 
    geom_point(size=3) + 
    xlab("Baseline [RSS]") + ylab("CpG [RSS]") + ggtitle("RSS") + 
    xlim(c(min(c(rss_mat$rss_baseline, rss_mat$rss_cpg)), max(c(rss_mat$rss_baseline, rss_mat$rss_cpg)))) + 
    ylim(c(min(c(rss_mat$rss_baseline, rss_mat$rss_cpg)), max(c(rss_mat$rss_baseline, rss_mat$rss_cpg)))) + 
    facet_wrap(~leiden_cell_type_main, scales="free_y", ncol=10) + 
    scale_color_manual(values=color$leiden_cell_type_main) + 
    theme(legend.position="none") 
    

# Session info

In [ ]:
sessionInfo()